In [17]:
using Revise
using QuantumOptimalControl
using QuantumOptics
using LinearAlgebra
using Flux
using PlotlyJS
using Random
using ProgressMeter
ProgressMeter.ijulia_behavior(:clear);

WebIO._IJuliaInit()

In [144]:
Δ = -0.3 * 2π
T = 4.0
σ = T / 4;

In [145]:
bs = NLevelBasis(3)

H0 = Δ * transition(bs, 3, 3)
Hx = transition(bs, 2, 1) + transition(bs, 1, 2) + √2 * (transition(bs, 3, 2) + transition(bs, 2, 3))
Hy = 1im * (transition(bs, 2, 1) - transition(bs, 1, 2) + √2 * (transition(bs, 3, 2) - transition(bs, 2, 3)));

In [146]:
Ωx(A, t) = A * (exp(-(t - T / 2)^2 / (2σ^2)) - exp(-T^2 / (8σ^2)))
Ωy(A, β, t) = -2 * (β / Δ) * A * (t - T / 2) * exp(-(t - T / 2)^2 / (2σ^2));

In [147]:
coeffs(p, t) = [Ωx(p[1], t) / 2, Ωy(p[1], p[2], t) / 2]
θ = [0.0, 0.0];

In [148]:
cost = CostFunction((x, y)-> 1.0 - abs2(x'*y));

In [149]:
trans = UnitaryTransform([nlevelstate(bs, 1), nlevelstate(bs, 2)], 
                         [[1.0 -1.0im];[-1.0im 1.0]]/√2);

In [150]:
tspan = (0.0, T)
H = Hamiltonian(H0, [Hx, Hy], coeffs)
prob = QOCProblem(H, trans, tspan, cost);

In [151]:
@time sol = solve(prob, θ, ADAM(0.01); maxiter=200);

Progress: 100%|█████████████████████████████████████████| Time: 0:01:23
  distance:     0.004422796950737717
  constraints:  0.0


 83.694219 seconds (10.51 M allocations: 546.765 MiB, 96.66% gc time, 1.80% compilation time)


In [152]:
plot(sol.distance_trace, Layout(yaxis_type="log"))

data: [
  "scatter with fields type, x, and y"
]

layout: "layout with fields margin, template, and yaxis"

In [153]:
ts = collect(0:T/100.0:T)
f= plot([
    scatter(x=ts, y=Ωx.(sol.params[1], ts), name="Ωx")
    scatter(x=ts, y=Ωy.(sol.params[1], sol.params[2], ts), name="Ωy")
    ],
    Layout(
        xaxis_title_text="Time (ns)",
        yaxis_title_text="Frequency (GHz)",
        legend=attr(x=0, y=1,),
        font=attr(
            size=16,
        )
    )
)

data: [
  "scatter with fields name, type, x, and y",
  "scatter with fields name, type, x, and y"
]

layout: "layout with fields font, legend, margin, template, xaxis, and yaxis"